# LightFM user features
全部入れると良くなかったので、一旦ユーザーの特徴のみ入れてみる

In [7]:
import datetime

import faiss
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import psutil
from lightfm import LightFM

import schema
from metric import mapk
from scipy import sparse

In [8]:
articles = pd.read_csv('input/transformed/articles.csv', usecols=schema.ARTICLES.keys(), dtype=schema.ARTICLES)
customers = pd.read_csv('input/transformed/customers.csv', usecols=schema.CUSTOMERS.keys(), dtype=schema.CUSTOMERS)
n_user = len(customers)
n_item = len(articles)

In [9]:
transactions = pd.read_csv('input/transformed/transactions_train.csv', parse_dates=['t_dat'], usecols=list(schema.TRANSACTIONS.keys())+['t_dat'], dtype=schema.TRANSACTIONS)
TOPK = 12

In [10]:
def create_customer_features(customers):
    df = customers.copy().drop('customer_id_idx', axis=1)
    df['age_is_null'] = df['age'].isnull().astype(int)
    df['age'] = df['age'].fillna(df['age'].mean())
    for c in ['club_member_status_idx', 'fashion_news_frequency_idx']:
        df = pd.concat([df, pd.get_dummies(df[c], prefix=c)], axis=1).drop(c, axis=1)

    idxs = sparse.identity(len(df), dtype='f')
    return sparse.hstack([idxs, df.values]).astype('float32')


def create_article_features(articles):
    df = articles.copy().drop('article_id_idx', axis=1)
    for c in df.columns:
        df = pd.concat([df, pd.get_dummies(df[c], prefix=c)], axis=1).drop(c, axis=1)

    idxs = sparse.identity(len(df), dtype='f')
    return sparse.hstack([idxs, df.values]).astype('float32')

user_features = create_customer_features(customers)
# item_features = create_article_features(articles)

In [11]:
def objective(trial: optuna.Trial) -> float:
    train_days = trial.suggest_int('train_days', 7, 28, 7)
    no_components = trial.suggest_int('no_components', 128, 1024, 128)
    learning_schedule = 'adadelta'
    loss = 'bpr'
    learning_rate = trial.suggest_loguniform('learning_rate', 5*1e-3, 5*1e-2)
    item_alpha = trial.suggest_loguniform('item_alpha', 1e-12, 1e-6)
    user_alpha = trial.suggest_loguniform('user_alpha', 1e-12, 1e-6)
    max_sampled = trial.suggest_int('max_sampled', 10, 20)
    epochs = 20

    lightfm_params = {
        'no_components': no_components,
        'learning_schedule': learning_schedule,
        'loss': loss,
        'learning_rate': learning_rate,
        'item_alpha': item_alpha,
        'user_alpha': user_alpha,
        'max_sampled': max_sampled,
    }
    print(lightfm_params)

    valid_start_date = datetime.date(2020, 9, 16)
    valid_end_date = datetime.date(2020, 9, 22)
    train_end_date = valid_start_date - datetime.timedelta(days=1)
    train_start_date = valid_start_date - datetime.timedelta(days=train_days)

    transactions_train = transactions.query("@train_start_date <= t_dat <= @train_end_date")
    transactions_valid = transactions.query("@valid_start_date <= t_dat <= @valid_end_date")

    val = transactions_valid.groupby('customer_id_idx')['article_id_idx'].apply(list).reset_index()

    train = sparse.lil_matrix((n_user, n_item))
    train[transactions_train.customer_id_idx, transactions_train.article_id_idx] = 1

    model = LightFM(**lightfm_params)
    model.fit(train, user_features=user_features, epochs=epochs, num_threads=psutil.cpu_count(logical=False), verbose=True)
    model.fit(train, epochs=epochs, num_threads=psutil.cpu_count(logical=False), verbose=True)

    index = faiss.index_factory(no_components, "Flat", faiss.METRIC_INNER_PRODUCT)
    index = faiss.index_cpu_to_gpu(faiss.StandardGpuResources(), 0, index)
    index.add(model.item_embeddings)
    _, idxs = index.search(model.user_embeddings, TOPK)

    return mapk(val.article_id_idx, idxs[val.customer_id_idx])

In [12]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, timeout=7200)

[I 2022-03-05 10:09:03,056] A new study created in memory with name: no-name-98108614-6651-42ad-9786-292c935b772f


{'no_components': 384, 'learning_schedule': 'adadelta', 'loss': 'bpr', 'learning_rate': 0.04097390085924968, 'item_alpha': 3.457096305467413e-07, 'user_alpha': 3.7218283401567005e-09, 'max_sampled': 20}


Epoch: 100%|██████████| 20/20 [00:30<00:00,  1.54s/it]
[I 2022-03-05 10:13:11,749] Trial 0 finished with value: 0.01866700699740334 and parameters: {'train_days': 21, 'no_components': 384, 'learning_rate': 0.04097390085924968, 'item_alpha': 3.457096305467413e-07, 'user_alpha': 3.7218283401567005e-09, 'max_sampled': 20}. Best is trial 0 with value: 0.01866700699740334.


{'no_components': 512, 'learning_schedule': 'adadelta', 'loss': 'bpr', 'learning_rate': 0.03349866242414398, 'item_alpha': 9.443054761010596e-11, 'user_alpha': 5.324836756187512e-09, 'max_sampled': 10}


Epoch: 100%|██████████| 20/20 [00:47<00:00,  2.37s/it]
[I 2022-03-05 10:19:24,492] Trial 1 finished with value: 0.019112590991056957 and parameters: {'train_days': 28, 'no_components': 512, 'learning_rate': 0.03349866242414398, 'item_alpha': 9.443054761010596e-11, 'user_alpha': 5.324836756187512e-09, 'max_sampled': 10}. Best is trial 1 with value: 0.019112590991056957.


...

Epoch: 100%|██████████| 20/20 [00:14<00:00,  1.34it/s]
[I 2022-03-05 11:58:30,211] Trial 17 finished with value: 0.016535167081349358 and parameters: {'train_days': 28, 'no_components': 128, 'learning_rate': 0.009645209238999959, 'item_alpha': 1.3087702307745533e-08, 'user_alpha': 2.5840362557649772e-11, 'max_sampled': 13}. Best is trial 13 with value: 0.019941548788653816.


{'no_components': 896, 'learning_schedule': 'adadelta', 'loss': 'bpr', 'learning_rate': 0.006646892788497398, 'item_alpha': 1.9214964137113685e-11, 'user_alpha': 6.167919643815403e-07, 'max_sampled': 16}


Epoch: 100%|██████████| 20/20 [01:08<00:00,  3.41s/it]
[I 2022-03-05 12:06:21,226] Trial 18 finished with value: 0.018183367842720826 and parameters: {'train_days': 28, 'no_components': 896, 'learning_rate': 0.006646892788497398, 'item_alpha': 1.9214964137113685e-11, 'user_alpha': 6.167919643815403e-07, 'max_sampled': 16}. Best is trial 13 with value: 0.019941548788653816.


{'no_components': 1024, 'learning_schedule': 'adadelta', 'loss': 'bpr', 'learning_rate': 0.008320283184731108, 'item_alpha': 2.1235303822142049e-10, 'user_alpha': 2.5716342100158757e-08, 'max_sampled': 10}


Epoch: 100%|██████████| 20/20 [01:04<00:00,  3.22s/it]
[I 2022-03-05 12:14:14,694] Trial 19 finished with value: 0.019661758952200868 and parameters: {'train_days': 21, 'no_components': 1024, 'learning_rate': 0.008320283184731108, 'item_alpha': 2.1235303822142049e-10, 'user_alpha': 2.5716342100158757e-08, 'max_sampled': 10}. Best is trial 13 with value: 0.019941548788653816.


In [13]:
study.trials_dataframe().sort_values(by='value', ascending=False).head(20)

,number,value,datetime_start,datetime_complete,duration,params_item_alpha,params_learning_rate,params_max_sampled,params_no_components,params_train_days,params_user_alpha,state
13,13,0.019942,2022-03-05 11:26:24.382729,2022-03-05 11:34:37.594407,0 days 00:08:13.211678,2.060122e-11,0.008768,17,896,28,1.568942e-12,COMPLETE
9,9,0.019723,2022-03-05 10:55:54.920487,2022-03-05 11:05:49.844129,0 days 00:09:54.923642,5.238870e-10,0.007170,11,1024,28,2.386060e-09,COMPLETE
19,19,0.019662,2022-03-05 12:06:21.227686,2022-03-05 12:14:14.694604,0 days 00:07:53.466918,2.123530e-10,0.008320,10,1024,21,2.571634e-08,COMPLETE
4,4,0.019645,2022-03-05 10:29:41.263655,2022-03-05 10:36:31.428736,0 days 00:06:50.165081,2.468518e-12,0.012563,15,896,21,6.329080e-12,COMPLETE
7,7,0.019617,2022-03-05 10:45:53.522395,2022-03-05 10:54:14.861605,0 days 00:08:21.339210,4.619953e-08,0.008821,11,1024,21,1.225197e-11,COMPLETE
15,15,0.019544,2022-03-05 11:44:42.250394,2022-03-05 11:52:24.084984,0 days 00:07:41.834590,9.791160e-10,0.005503,17,768,28,3.924055e-10,COMPLETE
3,3,0.019543,2022-03-05 10:24:06.849207,2022-03-05 10:29:41.262584,0 days 00:05:34.413377,1.140520e-10,0.006786,20,640,21,5.998556e-09,COMPLETE
11,11,0.019527,2022-03-05 11:09:54.026009,2022-03-05 11:18:15.541748,0 days 00:08:21.515739,8.588392e-12,0.014821,15,896,28,1.557944e-12,COMPLETE
12,12,0.019504,2022-03-05 11:18:15.543045,2022-03-05 11:26:24.381409,0 days 00:08:08.838364,3.204082e-09,0.013484,14,896,28,1.651820e-10,COMPLETE
14,14,0.019464,2022-03-05 11:34:37.595490,2022-03-05 11:44:42.249076,0 days 00:10:04.653586,2.445897e-11,0.007362,17,1024,28,4.558985e-08,COMPLETE


この実験では特徴いれないほうがよい